# LLaVA: Large Language and Vision Assistant

This notebook demonstrates the capabilities of LLaVA, a multimodal model that combines vision and language understanding based on the paper "Visual Instruction Tuning" (NeurIPS 2023).

LLaVA connects a vision encoder (CLIP) with a large language model (e.g., Vicuna) using a projection layer, enabling it to understand and reason about images.

## Setup

First, let's import the necessary libraries and set up the model.

In [ ]:
import os
import sys
import torch
from PIL import Image
import matplotlib.pyplot as plt
import requests
from io import BytesIO

# Add parent directory to path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from models.llava import LLaVA
from utils.visualization import display_image_with_caption

## Initialize the Model

Now, let's initialize the LLaVA model. This will download the necessary model weights if they're not already available.

In [ ]:
# Check if CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Initialize the model
# Note: For lower memory usage, you can set load_in_8bit=True or load_in_4bit=True
model = LLaVA(
    vision_model_path="openai/clip-vit-large-patch14-336",
    language_model_path="lmsys/vicuna-7b-v1.5",
    device=device,
    load_in_8bit=False  # Set to True if you have limited GPU memory
)

## Helper Functions

Let's define some helper functions to download images from URLs and display them with captions.

In [ ]:
def download_image(url):
    """Download an image from a URL."""
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert("RGB")

def save_image(image, filename):
    """Save an image to a file."""
    image.save(filename)
    return filename

def process_image_url(url, prompt, max_new_tokens=256, temperature=0.7):
    """Process an image from a URL and generate a response."""
    # Download and save the image
    image = download_image(url)
    filename = "temp_image.jpg"
    save_image(image, filename)
    
    # Generate response
    response = model.generate_from_image(
        image_path=filename,
        prompt=prompt,
        max_new_tokens=max_new_tokens,
        temperature=temperature
    )
    
    # Display the image and response
    plt.figure(figsize=(10, 8))
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"Prompt: {prompt}")
    plt.figtext(0.5, 0.01, f"Response: {response}", wrap=True, horizontalalignment='center', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    # Clean up
    if os.path.exists(filename):
        os.remove(filename)
    
    return response

## Example 1: Basic Image Description

Let's start with a simple example: asking the model to describe an image.

In [ ]:
image_url = "https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg"
prompt = "What's in this image?"

response = process_image_url(image_url, prompt)

## Example 2: Detailed Description

Now, let's ask for a more detailed description of the same image.

In [ ]:
prompt = "Describe this image in detail, including all the objects and their relationships."

response = process_image_url(image_url, prompt, max_new_tokens=512)

## Example 3: Visual Reasoning

Let's test the model's visual reasoning capabilities with a more complex image.

In [ ]:
image_url = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "What are the potential dangers or safety concerns in this location?"

response = process_image_url(image_url, prompt, max_new_tokens=384)

## Example 4: Object Counting

Let's test the model's ability to count objects in an image.

In [ ]:
image_url = "https://llava-vl.github.io/static/images/coco.jpg"
prompt = "How many people are in this image? Count carefully."

response = process_image_url(image_url, prompt)

## Example 5: Multi-turn Conversation

Let's demonstrate a multi-turn conversation about an image.

In [ ]:
image_url = "https://llava-vl.github.io/static/images/monalisa.jpg"
image = download_image(image_url)
filename = "temp_image.jpg"
save_image(image, filename)

# Display the image
plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.axis('off')
plt.title("Image for conversation")
plt.show()

# Initialize conversation
conversation = [
    {"role": "system", "content": "You are a helpful assistant that can understand images and answer questions about them."}
]

# First turn
prompt = "What is this famous artwork?"
response = model.generate_from_image(
    image_path=filename,
    prompt=prompt,
    conversation=conversation.copy()
)
print(f"User: {prompt}")
print(f"Assistant: {response}\n")

# Update conversation
conversation.append({"role": "user", "content": prompt})
conversation.append({"role": "assistant", "content": response})

# Second turn
prompt = "Who painted it and when?"
response = model.generate_from_image(
    image_path=filename,
    prompt=prompt,
    conversation=conversation.copy()
)
print(f"User: {prompt}")
print(f"Assistant: {response}\n")

# Update conversation
conversation.append({"role": "user", "content": prompt})
conversation.append({"role": "assistant", "content": response})

# Third turn
prompt = "What makes this painting so famous and unique?"
response = model.generate_from_image(
    image_path=filename,
    prompt=prompt,
    conversation=conversation.copy()
)
print(f"User: {prompt}")
print(f"Assistant: {response}")

# Clean up
if os.path.exists(filename):
    os.remove(filename)

## Conclusion

In this notebook, we've demonstrated the capabilities of LLaVA, a multimodal model that combines vision and language understanding. The model can:

1. Describe images at different levels of detail
2. Perform visual reasoning tasks
3. Count objects in images
4. Engage in multi-turn conversations about images

These capabilities make LLaVA useful for a wide range of applications, from image captioning to visual question answering and beyond.